In [4]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
# import project_heart as ph
from project_heart.modules.geometry import Geometry
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [5]:
filepath = Path("../sample_files/lvtetmesh_fibers_LDRB_1.vtk")
lv = LV.from_pyvista_read(filepath)

In [3]:
lv.create_nodesets_from_surfaces(mesh_data=LV_MESH_DATA.APEX_BASE_REGIONS, overwrite=False)
lv.create_nodesets_from_surfaces(mesh_data=LV_MESH_DATA.EPI_ENDO, overwrite=False)
lv.create_nodesets_from_surfaces(mesh_data=LV_MESH_DATA.SURFS, overwrite=False)
lv.create_nodesets_from_surfaces(mesh_data=LV_MESH_DATA.AM_SURFS, overwrite=False)
lv.create_nodesets_from_surfaces(mesh_data=LV_MESH_DATA.SURFS_DETAILED, overwrite=False)

In [4]:
lv.peek_unique_values_in_surface(LV_MESH_DATA.AM_SURFS, LV_SURFS)

['OTHER', 'AORTIC', 'MITRAL', 'AM_INTERCECTION']

In [5]:
lv.set_aortic_info()
lv.set_mitral_info()

In [7]:
aortic_bc = lv.create_spring_rim_bc("AORTIC_BC", LV_SURFS.BORDER_AORTIC)
mitral_bc = lv.create_spring_rim_bc("MITRAL_BC", LV_SURFS.BORDER_MITRAL)

In [9]:
atr_rim_string_lines = lv.get_rim_springs_for_plot(aortic_bc)
mtr_rim_string_lines = lv.get_rim_springs_for_plot(mitral_bc)

In [10]:
plotter = pv.Plotter(lighting='three lights')
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
plotter.add_points(lv.get_virtual_node(LV_VIRTUAL_NODES.MITRAL), color="green", point_size=300)
plotter.add_points(lv.get_virtual_node(LV_VIRTUAL_NODES.AORTIC), color="red", point_size=300)
plotter.add_points(aortic_bc[LV_RIM.NODES.value], color="red", point_size=300)
plotter.add_points(aortic_bc[LV_RIM.CENTER.value], color="red", point_size=300)

plotter.add_points(mitral_bc[LV_RIM.NODES.value], color="green", point_size=300)
plotter.add_points(mitral_bc[LV_RIM.CENTER.value], color="green", point_size=300)

plotter.add_mesh(atr_rim_string_lines, color="red", opacity=0.5, show_edges=False)
plotter.add_mesh(mtr_rim_string_lines, color="green", opacity=0.5, show_edges=False)
plotter.add_mesh(lv.mesh, 
                  scalars="LV_SURFS", 
                  cmap="Set2", 
                  opacity=1.0, 
                  show_edges=False,
                  ambient=0.2, 
                  diffuse=0.5, 
                  specular=0.5, 
                  specular_power=90,
                  #  smooth_shading=True,
                  )
plotter.enable_shadows()
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [11]:
lv.to_json("./lvtetmesh_wfibers_jsonsample.json", 
           nodeset_enums=[LV_SURFS],
           export_all_mesh_data=True)